In [2]:
import pandas as pd
df = pd.read_csv('csv/observations.csv')

In [4]:
df.columns

Index(['DATE', 'PATIENT', 'ENCOUNTER', 'CATEGORY', 'CODE', 'DESCRIPTION',
       'VALUE', 'UNITS', 'TYPE'],
      dtype='object')

In [7]:
df

,DATE,PATIENT,ENCOUNTER,CATEGORY,CODE,DESCRIPTION,VALUE,UNITS,TYPE
0,2019-02-17T05:07:38Z,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,748f8357-6cc7-551d-f31a-32fa2cf84126,vital-signs,8302-2,Body Height,51.4,cm,numeric
1,2019-02-17T05:07:38Z,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,748f8357-6cc7-551d-f31a-32fa2cf84126,vital-signs,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,1.0,{score},numeric
2,2019-02-17T05:07:38Z,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,748f8357-6cc7-551d-f31a-32fa2cf84126,vital-signs,29463-7,Body Weight,3.8,kg,numeric
3,2019-02-17T05:07:38Z,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,748f8357-6cc7-551d-f31a-32fa2cf84126,vital-signs,77606-2,Weight-for-length Per age and sex,57.9,%,numeric
4,2019-02-17T05:07:38Z,b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85,748f8357-6cc7-551d-f31a-32fa2cf84126,vital-signs,9843-4,Head Occipital-frontal circumference,34.8,cm,numeric
...,...,...,...,...,...,...,...,...,...
531139,2017-11-07T05:11:58Z,cb1b2c74-d1c5-997c-6f8b-20ca9f332eef,NaN,NaN,QOLS,QOLS,1.0,{score},numeric
531140,2018-11-07T05:11:58Z,cb1b2c74-d1c5-997c-6f8b-20ca9f332eef,NaN,NaN,QOLS,QOLS,1.0,{score},numeric
531141,2019-11-07T05:11:58Z,cb1b2c74-d1c5-997c-6f8b-20ca9f332eef,NaN,NaN,QOLS,QOLS,1.0,{score},numeric
531142,2020-11-07T05:11:58Z,cb1b2c74-d1c5-997c-6f8b-20ca9f332eef,NaN,NaN,QOLS,QOLS,1.0,{score},numeric


In [14]:
import pandas as pd
import os

def consolidate_observations_compact(input_filepath: str, output_filepath: str):
    """
    Reads an observations CSV, groups rows by encounter ID, includes the
    patient ID, and consolidates the information into a single, compact
    line of text for each encounter.

    Args:
        input_filepath (str): Path to the input observations.csv file.
        output_filepath (str): Path to save the consolidated CSV file.
    """
    print(f"Loading data from {input_filepath}...")
    try:
        # Load the CSV, ensuring PATIENT is included
        use_cols = ['PATIENT', 'ENCOUNTER', 'CATEGORY', 'DESCRIPTION', 'VALUE', 'UNITS']
        df = pd.read_csv(input_filepath, usecols=use_cols, low_memory=False)
        print("Data loaded successfully.")
    except (FileNotFoundError, ValueError) as e:
        print(f"Error loading file: {e}. Please check the file path and column names.")
        return

    # --- Step 1: Format each row into the new compact text snippet ---
    # Handle potential missing units and ensure correct data types
    df['UNITS'] = df['UNITS'].fillna('')
    df['VALUE'] = df['VALUE'].astype(str)
    df['DESCRIPTION'] = df['DESCRIPTION'].astype(str)
    df['CATEGORY'] = df['CATEGORY'].astype(str)

    # Create the compact snippet for each observation row
    # Format: (CATEGORY): DESCRIPTION = VALUE UNITS
    df['compact_snippet'] = (
        '(' + df['CATEGORY'] + '): ' +
        df['DESCRIPTION'] + ' = ' +
        df['VALUE'] + ' ' + df['UNITS'].str.strip()
    )
    print("Formatted individual observation rows into compact snippets.")

    # --- Step 2: Group by both ENCOUNTER and PATIENT, then aggregate ---
    print("Grouping by encounter and patient ID, then aggregating text...")

    # Group by both columns to keep the PATIENT ID.
    # The aggregation then joins the snippets with ", " for each group.
    consolidated = df.groupby(['ENCOUNTER', 'PATIENT'])['compact_snippet'].apply(', '.join).reset_index()

    # Then, add the "Observation " prefix to the entire aggregated string
    consolidated['compact_snippet'] = 'Observation ' + consolidated['compact_snippet']
    print("Aggregation complete.")

    # --- Step 3: Rename columns and save the result ---
    consolidated.rename(columns={
        'ENCOUNTER': 'encounter_id',
        'PATIENT': 'patient_id',
        'compact_snippet': 'text'
    }, inplace=True)

    # Reorder columns for the final output file
    final_df = consolidated[['patient_id', 'encounter_id', 'text']]

    try:
        # Ensure the output directory exists
        output_dir = os.path.dirname(output_filepath)
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
            
        final_df.to_csv(output_filepath, index=False)
        print(f"\n✅ Success! Processed {len(final_df)} unique encounters.")
        print(f"Consolidated data saved to {output_filepath}")
    except Exception as e:
        print(f"Error saving file: {e}")

# --- How to use the script ---
if __name__ == '__main__':
    # Define the path to your input and output files
    # Make sure the 'csv' directory exists and contains 'observations.csv'
    input_csv = 'csv/observations.csv'
    
    # The script will create the 'csv_new' directory if it doesn't exist
    output_csv = 'csv_new/consolidated_observations_compact.csv'

    consolidate_observations_compact(input_filepath=input_csv, output_filepath=output_csv)


Loading data from csv/observations.csv...
Data loaded successfully.
Formatted individual observation rows into compact snippets.
Grouping by encounter and patient ID, then aggregating text...
Aggregation complete.

✅ Success! Processed 21025 unique encounters.
Consolidated data saved to csv_new/consolidated_observations_compact.csv


In [15]:
df_new=pd.read_csv('csv_new/consolidated_observations_compact.csv')
df_new.head()

,patient_id,encounter_id,text
0,db0ba46f-9b5e-63cd-cbe4-3c0ea82abbdf,00067119-65e8-91d6-c8b5-4dcb2e8b55bf,Observation (laboratory): Glucose = 79.0 mg/dL...
1,066f03fb-a5aa-5b53-d4a6-b31f5b44a29f,00123b9a-aa74-43b1-fce6-8e8117016dae,Observation (vital-signs): Body Height = 158.4...
2,7673112e-f252-3c46-a7e7-ac1e1fe3fa45,0017333f-3c9d-6e96-e8fb-387bb51e9a91,Observation (vital-signs): Body Height = 160.1...
3,a47eed09-9942-9cbf-9220-2843ded300e9,00179b79-53a4-138d-e38d-4fd57bbb7564,Observation (vital-signs): Body Height = 140.3...
4,788a1414-0bee-f2a6-8096-a9f2d936f42f,00198a9d-085f-1890-67ad-49fbc95ba255,Observation (vital-signs): Body Height = 186.8...


In [16]:
medications = pd.read_csv('csv/medications.csv')
print(medications.columns)

Index(['START', 'STOP', 'PATIENT', 'PAYER', 'ENCOUNTER', 'CODE', 'DESCRIPTION',
       'BASE_COST', 'PAYER_COVERAGE', 'DISPENSES', 'TOTALCOST', 'REASONCODE',
       'REASONDESCRIPTION'],
      dtype='object')


In [17]:
import pandas as pd
import os

def consolidate_medications(input_filepath: str, output_filepath: str):
    """
    Reads medications.csv, groups by encounter, and concatenates descriptions.
    """
    print(f"--- Consolidating Medications from {input_filepath} ---")
    try:
        use_cols = ['PATIENT', 'ENCOUNTER', 'DESCRIPTION']
        df = pd.read_csv(input_filepath, usecols=use_cols, low_memory=False)
    except (FileNotFoundError, ValueError) as e:
        print(f"Error loading medications file: {e}")
        return None

    df['DESCRIPTION'] = df['DESCRIPTION'].astype(str)

    consolidated = df.groupby(['ENCOUNTER', 'PATIENT'])['DESCRIPTION'].apply(', '.join).reset_index()
    consolidated['text'] = 'Medication: ' + consolidated['DESCRIPTION']

    final_df = consolidated[['PATIENT', 'ENCOUNTER', 'text']].rename(columns={'PATIENT': 'patient_id', 'ENCOUNTER': 'encounter_id'})
    
    try:
        # Ensure the output directory exists
        output_dir = os.path.dirname(output_filepath)
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
            
        final_df.to_csv(output_filepath, index=False)
        print(f"✅ Saved consolidated medications to {output_filepath}\n")
    except Exception as e:
        print(f"Error saving file: {e}")

    return final_df


if __name__ == '__main__':
    # Define input directories and file paths
    input_dir = 'csv/'
    output_dir = 'csv_new/'
    
    medications_input_path = os.path.join(input_dir, 'medications.csv')
    medications_output_path = os.path.join(output_dir, 'consolidated_medications.csv')

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Process only the medications file
    consolidate_medications(medications_input_path, medications_output_path)



--- Consolidating Medications from csv/medications.csv ---
✅ Saved consolidated medications to csv_new/consolidated_medications.csv



In [18]:
df_procedures=pd.read_csv('csv/procedures.csv')
df_procedures.columns

Index(['START', 'STOP', 'PATIENT', 'ENCOUNTER', 'CODE', 'DESCRIPTION',
       'BASE_COST', 'REASONCODE', 'REASONDESCRIPTION'],
      dtype='object')

In [19]:
import pandas as pd
import os

def consolidate_procedures(input_filepath: str, output_filepath: str):
    """
    Reads procedures.csv, groups by encounter, and concatenates descriptions.
    """
    print(f"--- Consolidating Procedures from {input_filepath} ---")
    try:
        use_cols = ['PATIENT', 'ENCOUNTER', 'DESCRIPTION']
        df = pd.read_csv(input_filepath, usecols=use_cols, low_memory=False)
    except (FileNotFoundError, ValueError) as e:
        print(f"Error loading procedures file: {e}")
        return None

    df['DESCRIPTION'] = df['DESCRIPTION'].astype(str)

    consolidated = df.groupby(['ENCOUNTER', 'PATIENT'])['DESCRIPTION'].apply(', '.join).reset_index()
    consolidated['text'] = 'Procedure: ' + consolidated['DESCRIPTION']

    final_df = consolidated[['PATIENT', 'ENCOUNTER', 'text']].rename(columns={'PATIENT': 'patient_id', 'ENCOUNTER': 'encounter_id'})
    
    try:
        # Ensure the output directory exists
        output_dir = os.path.dirname(output_filepath)
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
            
        final_df.to_csv(output_filepath, index=False)
        print(f"✅ Saved consolidated procedures to {output_filepath}\n")
    except Exception as e:
        print(f"Error saving file: {e}")

    return final_df


if __name__ == '__main__':
    # Define input directories and file paths
    input_dir = 'csv/'
    output_dir = 'csv_new/'
    
    procedures_input_path = os.path.join(input_dir, 'procedures.csv')
    procedures_output_path = os.path.join(output_dir, 'consolidated_procedures.csv')

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Process only the procedures file
    consolidate_procedures(procedures_input_path, procedures_output_path)



--- Consolidating Procedures from csv/procedures.csv ---
✅ Saved consolidated procedures to csv_new/consolidated_procedures.csv



In [20]:
conditions = pd.read_csv('csv/conditions.csv')
# find the number of redundant rows
conditions.columns

Index(['START', 'STOP', 'PATIENT', 'ENCOUNTER', 'CODE', 'DESCRIPTION'], dtype='object')

In [22]:
import pandas as pd
import os

def consolidate_conditions(input_filepath: str, output_filepath: str):
    """
    Reads conditions.csv, groups by encounter, and concatenates descriptions.
    """
    print(f"--- Consolidating Conditions from {input_filepath} ---")
    try:
        use_cols = ['PATIENT', 'ENCOUNTER', 'DESCRIPTION']
        df = pd.read_csv(input_filepath, usecols=use_cols, low_memory=False)
    except (FileNotFoundError, ValueError) as e:
        print(f"Error loading conditions file: {e}")
        return None

    # --- Post-processing: Remove specified employment conditions ---
    conditions_to_remove = [
        'Full-time employment (finding)',
        'Part-time employment (finding)'
    ]
    initial_rows = len(df)
    df = df[~df['DESCRIPTION'].isin(conditions_to_remove)]
    print(f"Removed {initial_rows - len(df)} employment-related condition rows.")
    # -----------------------------------------------------------------

    df['DESCRIPTION'] = df['DESCRIPTION'].astype(str)

    consolidated = df.groupby(['ENCOUNTER', 'PATIENT'])['DESCRIPTION'].apply(', '.join).reset_index()
    consolidated['text'] = 'Condition: ' + consolidated['DESCRIPTION']

    final_df = consolidated[['PATIENT', 'ENCOUNTER', 'text']].rename(columns={'PATIENT': 'patient_id', 'ENCOUNTER': 'encounter_id'})
    
    try:
        # Ensure the output directory exists
        output_dir = os.path.dirname(output_filepath)
        if output_dir:
            os.makedirs(output_dir, exist_ok=True)
            
        final_df.to_csv(output_filepath, index=False)
        print(f"✅ Saved consolidated conditions to {output_filepath}\n")
    except Exception as e:
        print(f"Error saving file: {e}")

    return final_df


if __name__ == '__main__':
    # Define input directories and file paths
    input_dir = 'csv/'
    output_dir = 'csv_new/'
    
    conditions_input_path = os.path.join(input_dir, 'conditions.csv')
    conditions_output_path = os.path.join(output_dir, 'consolidated_conditions.csv')

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Process only the conditions file
    consolidate_conditions(conditions_input_path, conditions_output_path)



--- Consolidating Conditions from csv/conditions.csv ---
Removed 16231 employment-related condition rows.
✅ Saved consolidated conditions to csv_new/consolidated_conditions.csv

